In [1]:
import cv2
import mediapipe as mp
import pandas as pd

In [ ]:
""" FRONT ARMS RAISE HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []


with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [11, 12, 13, 14, 23, 24]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("FRONT ARMS RAISE HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("front_arms_raise_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
df_lm = pd.read_csv("front_arms_raise_landmarks.csv")
df_lm.head()

In [ ]:
""" SIDE ARMS RAISE HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []


with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [11, 12, 13, 14, 23, 24]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("SIDE ARMS RAISE HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("side_arms_raise_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


F:\Github Projects\MediRehab-AI-V2\data\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
df_lm = pd.read_csv("side_arms_raise_landmarks.csv")
df_lm.head()

,parts,x_coords,y_coords,z_coords,labels
0,LEFT_SHOULDER,0.761400,0.822606,-0.263205,ARMS RAISE
1,RIGHT_SHOULDER,0.485115,0.840813,-0.214504,ARMS RAISE
2,LEFT_SHOULDER,0.728986,0.811561,-0.263605,ARMS RAISE
3,RIGHT_SHOULDER,0.449961,0.846241,-0.208975,ARMS RAISE
4,LEFT_SHOULDER,0.734556,0.807241,-0.196921,ARMS RAISE


In [4]:
# df_parts_coords = df_lm.pivot(columns='parts', values=['x_coords', 'y_coords', 'z_coords'])
# df_parts_coords.head(n=20)
print(df_lm['parts'].value_counts())
#df_lm.loc[df_lm['parts'] == 'LEFT_SHOULDER', ['x_coords','y_coords','z_coords']].head()

parts
LEFT_SHOULDER     83
RIGHT_SHOULDER    83
RIGHT_ELBOW       24
Name: count, dtype: int64


In [ ]:
""" SQUAT HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [23, 24, 25, 26, 11, 12]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("SQUAT HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("squat_hold_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
df_lm = pd.read_csv("squat_hold_landmarks.csv")
df_lm.head()

In [ ]:
""" CALF RAISE HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [25, 26, 27, 28, 29, 30]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("CALF RAISE HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("calf_raise_hold_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
df_lm = pd.read_csv("calf_raise_hold_landmarks.csv")
df_lm.head()

In [ ]:
""" WALL SIT ARM HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [11, 12, 13, 14, 23, 24, 25, 26, 27, 28]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("WALL SIT ARM HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("wall_sit_arm_hold_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
df_lm = pd.read_csv("wall_sit_arm_hold_landmarks.csv")
df_lm.head()

In [ ]:
""" CHEST SQUEEZE HOLD """

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                if id in [11, 12, 13, 14, 23, 24]:
                    if not landmark.visibility > 0.8:
                        cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                        cv2.imshow('Pose Estimation', frame)
                        continue
                    
                    landmark_enum = mp_pose.PoseLandmark(id)
                    name = landmark_enum.name
                    x, y = int(landmark.x * w), int(landmark.y * h)
                    
                    parts.append(name)
                    x_coords.append(landmark.x)
                    y_coords.append(landmark.y)
                    z_coords.append(landmark.z)
                    labels.append("CHEST SQUEEZE HOLD")

                    cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)
        
        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("chest_squeeze_hold_landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
df_lm = pd.read_csv("chest_squeeze_hold_landmarks.csv")
df_lm.head()